Tras el fracaso del primer intento, tomamos las siguientes decisiones:

- Eliminar outliers.
- Clasificar la columna "Delay" en 5 categorías para reducir la cantidad de datos que tenemos.
- Eliminar las columnas "Positivos" y "Anonimos" puesto que la columna "Meneos" es la suma de ambas.

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Preprocessing
from sklearn.preprocessing import MinMaxScaler

# Train, Test
from sklearn.model_selection import train_test_split

# Preprocessing
from sklearn.preprocessing import MinMaxScaler

# Train, Test
from sklearn.model_selection import train_test_split

# Metricas para regresiones
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Regresores
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR

In [2]:
df = pd.read_csv("Data/MeneosCompleto.csv", delimiter=";")

pd.set_option("display.precision", 7)

df.head(3)

,Titular,Medio,URL,Enviado,Publicado,Meneos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma
0,Que arresten a los líderes de Israel por críme...,ctxt.es,https://old.meneame.net/story/arresten-lideres...,1715371648.0,1715379303.0,81,111.0,51.0,30.0,4.0,11.0,463.0
1,El pelotazo de los curas salesianos con una pi...,elsaltodiario.com,https://old.meneame.net/story/pelotazo-curas-s...,1715367709.0,1715377802.0,73,336.0,43.0,30.0,3.0,20.0,441.0
2,Países Bajos no participa en el segundo ensayo...,hoy.es,https://old.meneame.net/story/paises-bajos-no-...,1715374020.0,1715375703.0,123,1275.0,72.0,51.0,3.0,39.0,460.0


In [3]:
def limpieza_datos(df):
    #Limpieza de duplicados
    df.drop_duplicates()

    #Elimina las filas con valores NaN de las columnas que no son Clicks.
    lista_columnas = df.columns.to_list()
    lista_columnas.remove("Clicks")

    for columna in lista_columnas:
        lista_indices = df[df[columna].isna() == True].index.tolist()
        df = df.drop(index = lista_indices)

    # Creamos la columna delay que es el tiempo que transcurre entre que se envía el meneo y se publica
    df["Delay"] = df["Publicado"] - df["Enviado"]

    #Categorización de la columna Delay
    def delay(x):
        if x < 1:
            return("Menos de 1 hora")
        elif 1 <= x < 3:
            return("Entre 1 y 3 horas")
        elif 3 <= x < 6:
            return("Entre 3 y 6 horas")
        elif 6 <= x < 9:
            return("Entre 6 y 9 horas")
        else:
            return("Más de 9 horas")
        
    df["Delay_horas"] = df["Delay"]/3600
        
    df["Delay_horas"] = df["Delay_horas"].apply(lambda x: delay(x))
    
    #Eliminación de columnas innecesarias
    df = df.drop(columns=["Medio", "Positivos", "Anonimos", "Delay"], axis=1)
    
    # Separamos nuestro dataframe en uno de train, test y otro de predicción
    df_tt = df[df["Clicks"].isna() == False]

    df_pred = df[df["Clicks"].isna() == True]
    df_pred.to_csv("Data/Datos_Pred.csv", index=False)

    # Eliminación de outliers
    df_tt = df_tt[df_tt["Meneos"].between(50, 3000)]
    df_tt = df_tt[df_tt["Negativos"] <= 13]
    df_tt = df_tt[df_tt["Comentarios"].between(10, 150)]
    df_tt = df_tt[df_tt["Karma"].between(230, 700)]
    df_tt = df_tt[df_tt["Clicks"].between(263, 37946)]

    # Guardamos nuestro dataframe de train, test
    df_tt.to_csv("Data/Datos_TrainTest.csv", index=False)
    
    return(df_tt)

In [4]:
df_tt = limpieza_datos(df)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274376 entries, 0 to 274375
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Titular      274373 non-null  object 
 1   Medio        274376 non-null  object 
 2   URL          274367 non-null  object 
 3   Enviado      274370 non-null  float64
 4   Publicado    274370 non-null  float64
 5   Meneos       274376 non-null  int64  
 6   Clicks       193678 non-null  float64
 7   Positivos    274369 non-null  float64
 8   Anonimos     274369 non-null  float64
 9   Negativos    274369 non-null  float64
 10  Comentarios  274367 non-null  float64
 11  Karma        274369 non-null  float64
dtypes: float64(8), int64(1), object(3)
memory usage: 25.1+ MB


In [6]:
df.shape

(274376, 12)

In [7]:
#Empezamos eliminando duplicados
df.drop_duplicates()

,Titular,Medio,URL,Enviado,Publicado,Meneos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma
0,Que arresten a los líderes de Israel por críme...,ctxt.es,https://old.meneame.net/story/arresten-lideres...,1715371648.0,1715379303.0,81,111.0,51.0,30.0,4.0,11.0,463.0
1,El pelotazo de los curas salesianos con una pi...,elsaltodiario.com,https://old.meneame.net/story/pelotazo-curas-s...,1715367709.0,1715377802.0,73,336.0,43.0,30.0,3.0,20.0,441.0
2,Países Bajos no participa en el segundo ensayo...,hoy.es,https://old.meneame.net/story/paises-bajos-no-...,1715374020.0,1715375703.0,123,1275.0,72.0,51.0,3.0,39.0,460.0
3,Todos los israelíes están entrenados en hasbar...,reddit.com,https://old.meneame.net/story/todos-israelies-...,1715361389.0,1715375102.0,117,1256.0,69.0,48.0,5.0,24.0,410.0
4,Cuando Einstein llamó “fascistas” a los gobern...,cadtm.org,https://old.meneame.net/story/cuando-einstein-...,1715346981.0,1715373609.0,260,415.0,139.0,121.0,2.0,17.0,461.0
...,...,...,...,...,...,...,...,...,...,...,...,...
274371,Instalar Linux en iPod Mini | GPL Tarragona,gpltarragona.org,https://old.meneame.net/story/instalar-linux-e...,1133950966.0,1133953201.0,29,NaN,29.0,0.0,0.0,0.0,31.0
274372,Entrevista de El Mundo a Jose Antonio Marina,el-mundo.es,https://old.meneame.net/story/entrevista-de-el...,1133946888.0,1133951401.0,18,NaN,18.0,0.0,0.0,0.0,22.0
274373,Prova xfce 4.2.3!,xfce.org,https://old.meneame.net/story/prova-xfce-423,1133945900.0,1133947801.0,34,NaN,34.0,0.0,0.0,0.0,13.0
274374,Los 84 errores de noxtrum,pacoros.net,https://old.meneame.net/story/los-84-errores-d...,1133943563.0,1133943902.0,61,NaN,61.0,0.0,0.0,5.0,11.0


In [8]:
df.isna().sum()

Titular            3
Medio              0
URL                9
Enviado            6
Publicado          6
Meneos             0
Clicks         80698
Positivos          7
Anonimos           7
Negativos          7
Comentarios        9
Karma              7
dtype: int64

In [9]:
df.shape

(274376, 12)

In [10]:
#Eliminamos las filas con valores NaN de las columnas que no son Clicks.

lista_columnas = df.columns.to_list()
lista_columnas.remove("Clicks")

for columna in lista_columnas:
    lista_indices = df[df[columna].isna() == True].index.tolist()
    df = df.drop(index = lista_indices)

In [11]:
# Creamos la columna delay que es el tiempo que transcurre entre que se envía el meneo y se publica
df["Delay"] = df["Publicado"] - df["Enviado"]

In [12]:
# Una vez calculada esa columna, enviado y publicado no hacen falta

df = df.drop(columns=["Enviado", "Publicado"], axis=1)

In [13]:
df.head(3)

,Titular,Medio,URL,Meneos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma,Delay
0,Que arresten a los líderes de Israel por críme...,ctxt.es,https://old.meneame.net/story/arresten-lideres...,81,111.0,51.0,30.0,4.0,11.0,463.0,7655.0
1,El pelotazo de los curas salesianos con una pi...,elsaltodiario.com,https://old.meneame.net/story/pelotazo-curas-s...,73,336.0,43.0,30.0,3.0,20.0,441.0,10093.0
2,Países Bajos no participa en el segundo ensayo...,hoy.es,https://old.meneame.net/story/paises-bajos-no-...,123,1275.0,72.0,51.0,3.0,39.0,460.0,1683.0


In [14]:
df["Delay_horas"] = df["Delay"]/3600

df["Delay_horas"].unique()

array([ 2.12638889,  2.80361111,  0.4675    , ...,  9.79305556,
       20.38666667,  0.05638889])

In [15]:
df.head(3)

,Titular,Medio,URL,Meneos,Clicks,Positivos,Anonimos,Negativos,Comentarios,Karma,Delay,Delay_horas
0,Que arresten a los líderes de Israel por críme...,ctxt.es,https://old.meneame.net/story/arresten-lideres...,81,111.0,51.0,30.0,4.0,11.0,463.0,7655.0,2.1263889
1,El pelotazo de los curas salesianos con una pi...,elsaltodiario.com,https://old.meneame.net/story/pelotazo-curas-s...,73,336.0,43.0,30.0,3.0,20.0,441.0,10093.0,2.8036111
2,Países Bajos no participa en el segundo ensayo...,hoy.es,https://old.meneame.net/story/paises-bajos-no-...,123,1275.0,72.0,51.0,3.0,39.0,460.0,1683.0,0.4675000


In [ ]:
plt.figure(figsize=(12, 7))

sns.histplot(df["Delay_horas"].apply(np.log))

plt.show()

In [ ]:
px.histogram(df["Delay_horas"], marginal="box")

In [ ]:
px.histogram(df["Delay_horas"].apply(np.log), marginal="box")

In [ ]:
plt.figure(figsize=(12, 7))

sns.boxplot(df["Delay_horas"].apply(np.log))

plt.show()

In [22]:
df["Delay_horas"].describe()

count    274367.0000000
mean          6.5195869
std           7.9097874
min           0.0000000
25%           1.5216667
50%           3.7613889
75%           8.5483333
max         226.4994444
Name: Delay_horas, dtype: float64

In [23]:
def delay(x):
    if x < 1:
        return("Menos de 1 hora")
    elif 1 <= x < 3:
        return("Entre 1 y 3 horas")
    elif 3 <= x < 6:
        return("Entre 3 y 6 horas")
    elif 6 <= x < 9:
        return("Entre 6 y 9 horas")
    else:
        return("Más de 9 horas")

df["Delay_horas"] = df["Delay_horas"].apply(lambda x: delay(x))

In [24]:
df["Delay_horas"].value_counts()

Delay_horas
Entre 1 y 3 horas    69318
Más de 9 horas       64559
Entre 3 y 6 horas    59955
Menos de 1 hora      47719
Entre 6 y 9 horas    32816
Name: count, dtype: int64

In [25]:
df = df.drop(columns = ["Delay", "Positivos", "Anonimos"], axis = 1)

In [26]:
df["Medio"].value_counts()

Medio
eldiario.es                    10261
youtube.com                     8239
elpais.com                      7274
publico.es                      7010
elmundo.es                      6747
                               ...  
ncov.r6.no                         1
dosmaletas.blogcindario.com        1
nbcwashington.com                  1
skamasle.com                       1
reviews.zdnet.co.uk                1
Name: count, Length: 30614, dtype: int64

In [27]:
# Eliminamos la columna medio porque la dimensionalidad es demasiado grande

df = df.drop(columns=["Medio"], axis=1)

In [28]:
df.head()

,Titular,URL,Meneos,Clicks,Negativos,Comentarios,Karma,Delay_horas
0,Que arresten a los líderes de Israel por críme...,https://old.meneame.net/story/arresten-lideres...,81,111.0,4.0,11.0,463.0,Entre 1 y 3 horas
1,El pelotazo de los curas salesianos con una pi...,https://old.meneame.net/story/pelotazo-curas-s...,73,336.0,3.0,20.0,441.0,Entre 1 y 3 horas
2,Países Bajos no participa en el segundo ensayo...,https://old.meneame.net/story/paises-bajos-no-...,123,1275.0,3.0,39.0,460.0,Menos de 1 hora
3,Todos los israelíes están entrenados en hasbar...,https://old.meneame.net/story/todos-israelies-...,117,1256.0,5.0,24.0,410.0,Entre 3 y 6 horas
4,Cuando Einstein llamó “fascistas” a los gobern...,https://old.meneame.net/story/cuando-einstein-...,260,415.0,2.0,17.0,461.0,Entre 6 y 9 horas


In [29]:
# Separamos nuestro dataframe en uno de train, test y otro de predicción

df_tt = df[df["Clicks"].isna() == False]
df_tt.to_csv("Data/Datos_TrainTest.csv", index=False)

df_pred = df[df["Clicks"].isna() == True]
df_pred.to_csv("Data/Datos_Pred.csv", index=False)

## OUTLIERS ##

# Meneos #

In [30]:
def plot_numeric(df_tt, feature, transform=None):
    fig, ax = plt.subplots(1,2, figsize=(14,4))
    
    if transform == False or transform == None:
        sns.histplot(df[feature], ax=ax[0])
        sns.boxplot(x=df[feature], ax=ax[1])
    else:
        sns.histplot(df[feature].apply(transform), ax=ax[0])
        sns.boxplot(x=df[feature].apply(transform), ax=ax[1])
    

In [ ]:
plot_numeric(df_tt, "Meneos")

In [ ]:
plot_numeric(df_tt, "Meneos", transform=(np.log))

In [33]:
df_tt["Meneos"].describe()

count    193671.0000000
mean        566.0068208
std         429.0707078
min          10.0000000
25%         281.0000000
50%         444.0000000
75%         714.0000000
max       11559.0000000
Name: Meneos, dtype: float64

In [34]:
ric = 714-281

lim_inf = 281 - 1.5*ric
lim_sup = 714 + 1.5*ric

print(lim_inf, lim_sup)

-368.5 1363.5


In [ ]:
plt.figure(figsize = (10,6))

stats = df_tt["Meneos"].apply(np.log).describe()

median = stats["50%"]

Q1 = stats["25%"]
Q3 = stats["75%"]

ric = Q3 - Q1

lim_l = Q1 - ric*2
lim_r = Q3 + ric*2

sns.histplot(df_tt["Meneos"].apply(np.log))

plt.axvline(median, color = "black")
plt.axvline(lim_l, color="black", alpha = 0.5)
plt.axvline(lim_r, color="black", alpha = 0.5)

plt.show()

In [36]:
lim_l

3.773298083322068

In [37]:
df_tt.shape

(193671, 8)

In [38]:
df_tt = df_tt[df_tt["Meneos"].between(50, 3000)]

df_tt

,Titular,URL,Meneos,Clicks,Negativos,Comentarios,Karma,Delay_horas
0,Que arresten a los líderes de Israel por críme...,https://old.meneame.net/story/arresten-lideres...,81,111.0,4.0,11.0,463.0,Entre 1 y 3 horas
1,El pelotazo de los curas salesianos con una pi...,https://old.meneame.net/story/pelotazo-curas-s...,73,336.0,3.0,20.0,441.0,Entre 1 y 3 horas
2,Países Bajos no participa en el segundo ensayo...,https://old.meneame.net/story/paises-bajos-no-...,123,1275.0,3.0,39.0,460.0,Menos de 1 hora
3,Todos los israelíes están entrenados en hasbar...,https://old.meneame.net/story/todos-israelies-...,117,1256.0,5.0,24.0,410.0,Entre 3 y 6 horas
4,Cuando Einstein llamó “fascistas” a los gobern...,https://old.meneame.net/story/cuando-einstein-...,260,415.0,2.0,17.0,461.0,Entre 6 y 9 horas
...,...,...,...,...,...,...,...,...
193675,¿Un melenudo nueva imagen de Pantene?,https://old.meneame.net/story/melenudo-nueva-i...,1220,21493.0,4.0,113.0,605.0,Entre 3 y 6 horas
193678,En todo el Sistema Solar,https://old.meneame.net/story/en-todo-el-siste...,400,8567.0,0.0,18.0,654.0,Entre 1 y 3 horas
193680,Un matrimonio denuncia a la embajada en Roma p...,https://old.meneame.net/story/matrimonio-denun...,877,4867.0,1.0,51.0,682.0,Entre 1 y 3 horas
193686,Díaz Ferrán pulveriza 3.000 empleos en menos d...,https://old.meneame.net/story/diaz-ferran-pulv...,871,879.0,3.0,55.0,722.0,Entre 1 y 3 horas


# Negativos #

In [ ]:
plt.figure(figsize = (10,6))

stats = df_tt["Negativos"].describe()

median = stats["50%"]

Q1 = stats["25%"]
Q3 = stats["75%"]

ric = Q3 - Q1

lim_l = Q1 - ric*2
lim_r = Q3 + ric*2

sns.histplot(df_tt["Negativos"])

plt.axvline(median, color = "black")
plt.axvline(lim_l, color="black", alpha = 0.5)
plt.axvline(lim_r, color="black", alpha = 0.5)

plt.show()

In [40]:
df["Negativos"].describe()

count    274367.0000000
mean          4.3537269
std           8.5602412
min           0.0000000
25%           0.0000000
50%           2.0000000
75%           5.0000000
max         246.0000000
Name: Negativos, dtype: float64

In [ ]:
px.histogram(df_tt["Negativos"], marginal="box")

In [ ]:
px.histogram(df_tt["Negativos"].apply(np.log), marginal="box")

In [43]:
df_tt = df_tt[df_tt["Negativos"] <= 13]

df_tt.head(3)

,Titular,URL,Meneos,Clicks,Negativos,Comentarios,Karma,Delay_horas
0,Que arresten a los líderes de Israel por críme...,https://old.meneame.net/story/arresten-lideres...,81,111.0,4.0,11.0,463.0,Entre 1 y 3 horas
1,El pelotazo de los curas salesianos con una pi...,https://old.meneame.net/story/pelotazo-curas-s...,73,336.0,3.0,20.0,441.0,Entre 1 y 3 horas
2,Países Bajos no participa en el segundo ensayo...,https://old.meneame.net/story/paises-bajos-no-...,123,1275.0,3.0,39.0,460.0,Menos de 1 hora


# Comentarios # 

In [ ]:
px.histogram(df_tt["Comentarios"], marginal="box")

In [45]:
df["Comentarios"].describe()

count    274367.0000000
mean         74.3390167
std          70.1521330
min           0.0000000
25%          31.0000000
50%          54.0000000
75%          93.0000000
max        2718.0000000
Name: Comentarios, dtype: float64

In [46]:
stats = df_tt["Comentarios"].describe()

ric = stats["75%"] - stats["25%"]

lim_inf = stats["25%"] - 0.1*ric
lim_sup = stats["75%"] + 1.5*ric

print(lim_inf, lim_sup)

31.3 205.5


In [47]:
df_tt = df_tt[df_tt["Comentarios"].between(10, 150)]

df_tt.shape

(153136, 8)

# Karma #

In [ ]:
px.histogram(df_tt["Karma"], marginal="box")

In [49]:
stats = df_tt["Karma"].describe()

ric = stats["75%"] - stats["25%"]

lim_inf = stats["25%"] - 1*ric
lim_sup = stats["75%"] + 1*ric

print(lim_inf, lim_sup)

185.0 710.0


In [50]:
stats

count    153136.0000000
mean        452.8340625
std         122.0615586
min          35.0000000
25%         360.0000000
50%         444.0000000
75%         535.0000000
max        6937.0000000
Name: Karma, dtype: float64

In [51]:
df_tt = df_tt[df_tt["Karma"].between(230, 700)]

df_tt.shape

(148764, 8)

## Clicks ##

In [ ]:
px.histogram(df_tt["Clicks"].apply(np.log), marginal="box")

In [53]:
lim_inf_clicks = np.exp(5.572154)

lim_sup_clicks = np.exp(10.54392)

print(lim_inf_clicks, lim_sup_clicks)

262.99999153724815 37946.0237001653


In [54]:
df_tt = df_tt[df_tt["Clicks"].between(263, 37946)]

df_tt.head()

,Titular,URL,Meneos,Clicks,Negativos,Comentarios,Karma,Delay_horas
1,El pelotazo de los curas salesianos con una pi...,https://old.meneame.net/story/pelotazo-curas-s...,73,336.0,3.0,20.0,441.0,Entre 1 y 3 horas
2,Países Bajos no participa en el segundo ensayo...,https://old.meneame.net/story/paises-bajos-no-...,123,1275.0,3.0,39.0,460.0,Menos de 1 hora
3,Todos los israelíes están entrenados en hasbar...,https://old.meneame.net/story/todos-israelies-...,117,1256.0,5.0,24.0,410.0,Entre 3 y 6 horas
4,Cuando Einstein llamó “fascistas” a los gobern...,https://old.meneame.net/story/cuando-einstein-...,260,415.0,2.0,17.0,461.0,Entre 6 y 9 horas
5,"Vetiver: conoce qué es, para qué sirve, usos y...",https://old.meneame.net/story/vetiver-conoce-s...,64,595.0,1.0,34.0,436.0,Más de 9 horas


## PREPROCESAMIENTO ##

In [57]:
df_tt.head(3)

,Titular,URL,Meneos,Clicks,Negativos,Comentarios,Karma,Delay_horas
1,El pelotazo de los curas salesianos con una pi...,https://old.meneame.net/story/pelotazo-curas-s...,73,336.0,3.0,20.0,441.0,Entre 1 y 3 horas
2,Países Bajos no participa en el segundo ensayo...,https://old.meneame.net/story/paises-bajos-no-...,123,1275.0,3.0,39.0,460.0,Menos de 1 hora
3,Todos los israelíes están entrenados en hasbar...,https://old.meneame.net/story/todos-israelies-...,117,1256.0,5.0,24.0,410.0,Entre 3 y 6 horas


In [58]:
params = {"Menos de 1 hora" : 0,
            "Entre 1 y 3 horas" : 1,
            "Entre 3 y 6 horas" : 2,
            "Entre 6 y 9 horas" : 3,
            "Más de 9 horas" : 4}

df_tt["Delay_labels"] = df_tt["Delay_horas"].replace(params)

df_tt.head()

C:\Users\deepi\AppData\Local\Temp\ipykernel_9184\3928429369.py:7: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



,Titular,URL,Meneos,Clicks,Negativos,Comentarios,Karma,Delay_horas,Delay_labels
1,El pelotazo de los curas salesianos con una pi...,https://old.meneame.net/story/pelotazo-curas-s...,73,336.0,3.0,20.0,441.0,Entre 1 y 3 horas,1
2,Países Bajos no participa en el segundo ensayo...,https://old.meneame.net/story/paises-bajos-no-...,123,1275.0,3.0,39.0,460.0,Menos de 1 hora,0
3,Todos los israelíes están entrenados en hasbar...,https://old.meneame.net/story/todos-israelies-...,117,1256.0,5.0,24.0,410.0,Entre 3 y 6 horas,2
4,Cuando Einstein llamó “fascistas” a los gobern...,https://old.meneame.net/story/cuando-einstein-...,260,415.0,2.0,17.0,461.0,Entre 6 y 9 horas,3
5,"Vetiver: conoce qué es, para qué sirve, usos y...",https://old.meneame.net/story/vetiver-conoce-s...,64,595.0,1.0,34.0,436.0,Más de 9 horas,4


In [59]:
modelos = [LinearRegression(),
            KNeighborsRegressor(),
            RadiusNeighborsRegressor(),
            DecisionTreeRegressor(),
            RandomForestRegressor(),
            SVR(),
            AdaBoostRegressor(),
            GradientBoostingRegressor()]

In [61]:
# train, test split

X = df_tt.iloc[:, 2:].drop(["Clicks", "Delay_horas"], axis = 1)
y = df_tt["Clicks"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

x_scaler = MinMaxScaler()
X_train = x_scaler.fit_transform(X_train)
X_test = x_scaler.transform(X_test)

y_train = y_train.values.reshape(-1)
y_test = y_test.values.reshape(-1)

# y_scaler = MinMaxScaler()
# y_train = y_scaler.fit_transform(y_train)
# y_test = y_scaler.transform(y_test)

datos_modelos = list()

for model in modelos:

    model.fit(X_train, y_train)
    yhat = model.predict(X_test)

    # y_test_inv = y_scaler.inverse_transform(y_test.reshape(-1, 1))
    # yhat_inv = y_scaler.inverse_transform(yhat.reshape(-1, 1))

    #Metricas
    mae = mean_absolute_error(y_test, yhat)
    mse = mean_squared_error(y_test, yhat)
    r2 = r2_score(y_test, yhat)

    datos_modelos.append([str(model).strip("()"), model, mae, mse, r2])

df_modelo = pd.DataFrame(data = datos_modelos, columns = ["name", "model", "mae", "mse", "r2"])

df_modelo.sort_values("r2", ascending = False)



,name,model,mae,mse,r2
7,GradientBoostingRegressor,([DecisionTreeRegressor(criterion='friedman_ms...,2827.9553974,1.6965136e+07,0.1752962
0,LinearRegression,LinearRegression(),2950.5849943,1.7866804e+07,0.1314645
4,RandomForestRegressor,"(DecisionTreeRegressor(max_features=1.0, rando...",2979.3863078,1.8306857e+07,0.1100729
2,RadiusNeighborsRegressor,RadiusNeighborsRegressor(),3161.1399709,2.0177475e+07,0.0191389
1,KNeighborsRegressor,KNeighborsRegressor(),3093.8661825,2.0429906e+07,0.0068679
5,SVR,SVR(),2773.3792699,2.0843156e+07,-0.0132209
6,AdaBoostRegressor,"(DecisionTreeRegressor(max_depth=3, random_sta...",3898.6937745,2.3589310e+07,-0.1467161
3,DecisionTreeRegressor,DecisionTreeRegressor(),3989.7835048,3.6505250e+07,-0.7745817
